# SmartBeds - Manifolds

## Grado en Ingeniería Informática
## Universidad de Burgos
##### José Luis Garrido Labrador

In [1]:
%matplotlib inline 
#para dibujar en el propio notebook
import pandas as pd # se importa pandas como pd
import numpy as np  #numpy como np
import matplotlib.pyplot as plt #%matplotlilib.pyplot como plt
import mpl_toolkits.mplot3d #Matplotlib 3d
import pickle as pk #Pikcle as pk
import gc #Garbage colector

## Lectura de datos

In [2]:
with open('datos.pdd','rb') as f:
    datos = pk.load(f)
    
datos_seizure = datos.loc[datos['target']==True]
datos_notSeizure = datos.loc[datos['target']==False]

In [22]:
with open('datosE.pdd','rb') as f:
    dataE = pk.load(f)

dataE_seizure = dataE.loc[dataE['target']==True]
dataE_notSeizure = dataE.loc[dataE['target']==False]

## Particionado - Solo día de ataques

In [5]:
### Días donde hubo crisis
dayOfSeizures = set()

for i in datos_seizure['DateTime']:
    dayOfSeizures.add(i.date())


#Todos los datos de esos días
crit = datos['DateTime'].map(lambda x: x.date() in dayOfSeizures)

datosPart = datos[crit]
datosPart.head()
len(datosPart)

118138

## SpectralEmbedding

In [3]:
from sklearn.manifold import SpectralEmbedding as SE

### Datos brutos

In [6]:
data = datosPart.iloc[:,1:13]

In [7]:
seB = SE(n_components=2,n_neighbors=50).fit_transform(data) #Con 50 vecinos no peta

In [ ]:
plt.figure(figsize=(24,24))
seSample = seB

target = datosPart['target'].values
notTarget = np.logical_not(target)

serie = pd.DataFrame(seSample)

X = pd.Series(serie[0]).loc[notTarget]
Y = pd.Series(serie[1]).loc[notTarget]

X_t = pd.Series(serie[0]).loc[target]
Y_t = pd.Series(serie[1]).loc[target]

plt.scatter(X,Y,s=1)
plt.scatter(X_t,Y_t,s=1,c='r')
plt.title('Spectral Embedding Raw - Days of seizures')


Text(0.5, 1.0, 'Spectral Embedding Raw - Days of seizures')